In [2]:
import numpy as np
import time
from abc import ABC, abstractmethod
from typing import List, Optional

## Utils

In [3]:
def int2bin(x: int, size: int):
    return bin(x)[2:].zfill(size)


def float2int(x: float, n: int):
    return int(x * pow(2, n))

## Bit Stream

In [4]:
class BitStream:
    def __init__(self):
        self.stream = 0  # The content of the stream
        self.size = 0  # The current size of the stream

    def add_bits(self, bits: int, bit_count: int):
        """Add bits to the stream.

        Args:
            bits (int): The integer containing the bits to add (e.g.  0b101).
            bit_count (int): The number of bits to add from the 'bits' integer (e.g. 3).
        """
        self.stream = (self.stream << bit_count) | (bits & ((1 << bit_count) - 1))
        self.size += bit_count

    def pop_bits(self, bit_count: int) -> int:
        """Read qnd remove a certain number of bits from the stream.

        Args:
            bit_count (int): The number of bits to read.

        Returns:
            int: The extracted bits as an integer.
        """
        if bit_count > self.size:
            raise ValueError("Not enough bits in the stream to read.")

        # Extract the bits from the most significant part of the stream
        result = self.stream >> (self.size - bit_count)
        # Remove the bits we just read
        self.stream &= (1 << (self.size - bit_count)) - 1
        self.size -= bit_count

        return result

    def read(self):
        """Read the content of the stream as a binary int

        Returns:
            int: the content of the stream as an integer
        """
        return self.stream

    def __str__(self) -> str:
        """Return the bit stream as a binary string for easy visualization."""
        return bin(self.stream)[2:].zfill(self.size)

In [5]:
bitStream = BitStream()
bitStream.add_bits(0b00010, 5)
print(bin(bitStream.stream)[2:].zfill(5))
bitStream.pop_bits(2)
print(bin(bitStream.stream)[2:].zfill(2))

00010
10


## LFSR

In [6]:
class LFSR:
    def __init__(self, seed: int, taps: list[int], size: int):
        self.seed = seed
        self.taps = taps
        self.size = size
        self.register = seed
        self.output_bit = 0

    def step(self):
        feedback = 0
        for tap in self.taps:
            feedback ^= (self.register >> tap) & 1

        # Shift the register to the right and set the most significant bit to the feedback bit
        self.register = (self.register >> 1) | (feedback << (self.size - 1))
        self.output_bit = self.register & 1
        return self.register

## Abstract Module

In [242]:
class SCModule(ABC):
    def __init__(self, input_modules: Optional[List["SCModule"]] = None, **kwargs):
        self.input_modules = input_modules
        if self.input_modules is not None:
            self.input_number = len(input_modules)
            self.input_bits = [0] * self.input_number
        self.output_bit = 0

    def step(self):
        if self.input_modules is not None:
            for i in range(self.input_number):
                self.input_bits[i] = self.input_modules[i].output_bit

# B2S and S2B convertor

In [243]:
class Binary2Stochastic(SCModule):
    def __init__(self, lfsr_seed: int, lfsr_taps: int, lfsr_size: int):
        super().__init__()
        self.lfsr = LFSR(seed=lfsr_seed, taps=lfsr_taps, size=lfsr_size)
        self.n = lfsr_size

    def set_input_value(self, input_value: float):
        self.input_value = input_value

    def step(self):
        super().step()
        assert 0 < self.input_value < 1, "Input needs to be in range [0,1]"
        x_as_binary = float2int(self.input_value, self.n)
        rng = self.lfsr.step()
        self.output_bit = int(x_as_binary > rng)

In [244]:
class Stochastic2Binary(SCModule):
    def __init__(self, input_modules: Optional[List[SCModule]] = None):
        assert (
            len(input_modules) < 2
        ), "Stochastic to Binary module cannot have more than one entry"
        super().__init__(input_modules)
        self.clock = 0
        self.accumulator = 0
        self.result = 0

    def step(self):
        super().step()
        self.accumulator += self.input_bits[0]
        self.clock += 1
        self.result = self.accumulator / self.clock
        self.input_modules[0].step()

    def reset_clock(self):
        self.clock = 0
        self.accumulator = 0
        self.result = 0

## Multiplicator

In [1]:
class Multiplicator(SCModule):
    def __init__(self, input_modules: List[SCModule] = None):
        assert (
            len(input_modules) == 2
        ), "Multiplicator module must have 2 and exactly 2 entries"
        super().__init__(input_modules=input_modules)

    def step(self):
        super().step()
        self.output_bit = (self.input_bits[0] & self.input_bits[1]) | (
            ~self.input_bits[0] & ~self.input_bits[1]
        )
        for input_module in self.input_modules:
            input_module.step()

NameError: name 'SCModule' is not defined

## Adder

In [ ]:
class Adder(SCModule):
    def __init__(
        self,
        lfsr_seed: int,
        lfsr_taps: int,
        lfsr_size: int,
        input_modules: List[SCModule] = None,
    ):
        assert len(input_modules) == 2, "Adder module must have 2 and exactly 2 entries"
        super().__init__(input_modules=input_modules)
        self.lfsr = LFSR(lfsr_seed, lfsr_taps, lfsr_size)

    def step(self):
        super().step()
        self.lfsr.step()
        self.output_bit = self.input_bits[self.lfsr.output_bit]
        for input_module in self.input_modules:
            input_module.step()


In [252]:
main_seed1 = int(np.random.random() * (pow(2, 128) - 1))
main_seed2 = int(np.random.random() * (pow(2, 128) - 1))
main_seed3 = int(np.random.random() * (pow(2, 128) - 1))
main_taps = [0, 1, 2, 7]

In [253]:
b2s1 = Binary2Stochastic(lfsr_seed=main_seed1, lfsr_taps=main_taps, lfsr_size=128)
b2s2 = Binary2Stochastic(lfsr_seed=main_seed2, lfsr_taps=main_taps, lfsr_size=128)
mul = Multiplicator(input_modules=[b2s1, b2s2])
add = Adder(
    lfsr_seed=main_seed3, lfsr_taps=main_taps, lfsr_size=128, input_modules=[b2s1, b2s2]
)
s2b = Stochastic2Binary(input_modules=[add])

b2s1.set_input_value(0.3)
b2s2.set_input_value(0.5)

output_len = 100000
for _ in range(output_len):
    s2b.step()

print(s2b.result)

0.40552
